In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, 3) # Input = (1 * 28 * 28), Output = (26 * 26 * 8)
        self.batchnorm1 = nn.BatchNorm2d(8) # Input (26 * 26 * 8), Output (26 * 26 * 8)

        self.conv2 = nn.Conv2d(8, 16, 3) # Input = (26 * 26 * 16), Output = (24 * 24 * 16)
        self.batchnorm2 = nn.BatchNorm2d(16) # Input (24 * 24 * 16), Output (24 * 24 * 16)

        self.conv3 = nn.Conv2d(16, 16, 3) # Input = (24 * 24 * 16), Output = (22 * 22 * 16)
        self.batchnorm3 = nn.BatchNorm2d(16) # Input (22 * 22 * 16), Output (22 * 22 * 16)

        self.pool1 = nn.MaxPool2d(2, 2) #Input = (22 * 22 * 16), Output = (11 * 11 * 16)

        self.conv4 = nn.Conv2d(16, 16, 3) # Input = (11 * 11 * 16), Output = (9 * 9 * 16)
        self.batchnorm4 = nn.BatchNorm2d(16) # Input (9 * 9 * 16), Output (9 * 9 * 16)

        self.conv5 = nn.Conv2d(16, 16, 3) # Input = (9 * 9 * 16), Output = (7 * 7 * 16)
        self.batchnorm5 = nn.BatchNorm2d(16) # Input (7 * 7 * 16), Output (7 * 7 * 16)

        self.conv6 = nn.Conv2d(16, 16, 3) # Input = (7 * 7 * 16), Output = (5 * 5 * 16)
        self.batchnorm6 = nn.BatchNorm2d(16) # Input (5 * 5 * 16), Output (5 * 5 * 16)

        self.conv7 = nn.Conv2d(16, 32, 3) # Input = (5 * 5 * 16), Output = (3 * 3 * 32)
        self.batchnorm7 = nn.BatchNorm2d(32) # Input (3 * 3 * 32), Output (3 * 3 * 32)

        self.conv8 = nn.Conv2d(32, 10, 1) # Input = (3 * 3 * 32), Output = (3 * 3 * 10)

        self.pool2 = nn.AvgPool2d(2) # Input = (3 * 3 * 10), Output = (1 * 1 * 10)

        self.fc = nn.Linear(in_features=1 * 1 * 10, out_features=10)

        self.dropout = nn.Dropout(0.05)

    def forward(self, x):
        x = self.batchnorm1(F.relu(self.conv1(x)))
        x = self.batchnorm2(F.relu(self.conv2(x)))
        x = self.batchnorm3(F.relu(self.conv3(x)))
        x = self.pool1(x)
        x = self.dropout (x)

        x = self.batchnorm4(F.relu(self.conv4(x)))
        x = self.batchnorm5(F.relu(self.conv5(x)))
        x = self.batchnorm6(F.relu(self.conv6(x)))
        x = self.dropout (x)
        x = self.batchnorm7(F.relu(self.conv7(x)))

        x = self.conv8(x)
        x = self.pool2(x)

        x = x.view(-1, 10)

        self.fc = nn.Linear(10, 10)

        return F.log_softmax(x)

In [3]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 26, 26]              80
       BatchNorm2d-2            [-1, 8, 26, 26]              16
            Conv2d-3           [-1, 16, 24, 24]           1,168
       BatchNorm2d-4           [-1, 16, 24, 24]              32
            Conv2d-5           [-1, 16, 22, 22]           2,320
       BatchNorm2d-6           [-1, 16, 22, 22]              32
         MaxPool2d-7           [-1, 16, 11, 11]               0
           Dropout-8           [-1, 16, 11, 11]               0
            Conv2d-9             [-1, 16, 9, 9]           2,320
      BatchNorm2d-10             [-1, 16, 9, 9]              32
           Conv2d-11             [-1, 16, 7, 7]           2,320
      BatchNorm2d-12             [-1, 16, 7, 7]              32
    

<ipython-input-2-a523668b3877>:55: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [4]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw



In [5]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Epoch:{} Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(epoch,
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [6]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 20):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]<ipython-input-2-a523668b3877>:55: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
loss=0.054516177624464035 batch_id=468: 100%|██████████| 469/469 [00:20<00:00, 23.15it/s]



Test set: Epoch:1 Average loss: 0.0607, Accuracy: 9821/10000 (98%)



loss=0.06605575233697891 batch_id=468: 100%|██████████| 469/469 [00:16<00:00, 28.22it/s]



Test set: Epoch:2 Average loss: 0.0349, Accuracy: 9892/10000 (99%)



loss=0.028613736853003502 batch_id=468: 100%|██████████| 469/469 [00:16<00:00, 28.32it/s]


...


Test set: Epoch:17 Average loss: 0.0213, Accuracy: 9928/10000 (99%)



loss=0.0163444671779871 batch_id=468: 100%|██████████| 469/469 [00:16<00:00, 28.03it/s]



Test set: Epoch:18 Average loss: 0.0237, Accuracy: 9932/10000 (99%)



loss=0.005127995740622282 batch_id=468: 100%|██████████| 469/469 [00:16<00:00, 27.88it/s]



Test set: Epoch:19 Average loss: 0.0256, Accuracy: 9918/10000 (99%)

